In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_set = pd.read_csv('/kaggle/input/unimelb/unimelb_training.csv')
test_set = pd.read_csv('/kaggle/input/unimelb/unimelb_test.csv')

In [ ]:
## Copy file for edit

train = train_set.copy()
test = test_set.copy()

In [ ]:
train.head(5)

In [ ]:
## Check type of data

train.dtypes

In [ ]:
## Check column

train.columns

In [ ]:
## choose except feature

feature_except = ['Grant.Application.ID', 'Sponsor.Code', 'Start.date', 'RFCD.Code.1', 'RFCD.Percentage.1'
                  , 'RFCD.Code.2', 'RFCD.Percentage.2', 'RFCD.Code.3', 'RFCD.Percentage.3'
                  , 'RFCD.Code.4', 'RFCD.Percentage.4', 'RFCD.Code.5', 'RFCD.Percentage.5'
                  , 'Sponsor.Code', 'C.15', 'Unnamed: 251']

for col in range(1, 16):
    if 'No..of.Years.in.Uni.at.Time.of.Grant.'+ str(col) in list(train.columns):
        feature_except.append('No..of.Years.in.Uni.at.Time.of.Grant.'+str(col))
        
## Remove Nan data if more tahn 3000

for col in list(train.columns):
    if train[col].isnull().sum() > 3000:
        feature_except.append(col)

In [ ]:
## Select and drop feature for X and y

train = train.drop(feature_except, axis=1)

test = test.drop(feature_except, axis=1)

In [ ]:
train.head(5)

In [ ]:
## fill Nan in train set

for col in list(train.columns):
    if train[col].dtypes == 'object':
        train[col] = train[col].fillna(train[col].value_counts().idxmax())
    elif train[col].dtypes == 'float64':
        train[col] = train[col].fillna(train[col].mean())
    else:
        train[col] = train[col].fillna(train[col].max())

In [ ]:
## fill Nan in test set

for col in list(test.columns):
    if test[col].dtypes == 'object':
        test[col] = test[col].fillna(test[col].value_counts().idxmax())
    if test[col].dtypes == 'float64':
        test[col] = test[col].fillna(test[col].mean())
    else:
        test[col] = test[col].fillna(test[col].max())

In [ ]:
test.head(5)

In [ ]:
test.isnull().sum()

In [ ]:
## Separate to X and y

y = train['Grant.Status']
X = train.drop(['Grant.Status'], axis=1)

test = test.drop(['Grant.Status'], axis=1)

In [ ]:
## Transform to Encoder

from sklearn.preprocessing import LabelEncoder

## Transform to Encoder in train set

for col in list(X.columns):
    if X[col].dtypes == 'object':
        X[col] = LabelEncoder().fit_transform(X[col])

In [ ]:
## Transform to Encoder in test set

for col in list(test.columns):
    if test[col].dtypes == 'object':
        test[col] = LabelEncoder().fit_transform(test[col])

In [ ]:
X.head(5)

In [ ]:
## Splite data to train and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
## Create Root mean squar error

from sklearn.metrics import mean_squared_error

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
## Creating model RandomForest

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

model = RandomForestClassifier(n_estimators=5000, max_depth=5, random_state=0)
model.fit(X_train, y_train)
random_forest_Pred = model.predict(X_test)

print(rmsle(y_test, random_forest_Pred))

In [ ]:
## Creating model GradientBoostingClassifier

grad_boost = GradientBoostingClassifier(n_estimators=1000)
grad_boost.fit(X_train, y_train)
Y_pred = grad_boost.predict(X_test)

print(rmsle(y_test, Y_pred))

In [ ]:
## Creating model KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
Y_pred_2 = neigh.predict(X_test)

print(rmsle(y_test, Y_pred_2))


In [ ]:
## Creating model SVC

from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
Y_pred_3 = svc.predict(X_test)

print(rmsle(y_test, Y_pred_3))

In [ ]:
## Creating model DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier

decison_tree = DecisionTreeClassifier()
decison_tree.fit(X_train, y_train)
Y_pred_4 = decison_tree.predict(X_test)

print(rmsle(y_test, Y_pred_4))

In [ ]:
y = grad_boost.predict(test)

predic_y = pd.read_csv('/kaggle/input/unimelb/unimelb_example.csv')

del predic_y["Grant.Status"]

predic_y["Grant.Status"] = y

print(y)